# Part 2: Data preparation

One of the things we want to do here is to label the different directions into which the PELF can go from the crossover point (50,50). We will call these directions `N`, `E`, `S` and `W` for North, East, South and West. We will use the first calculated angle after the crossover point for this task. <br>

In [ ]:
import os
import pickle
import random
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
DATAPATH_INTERMEDIATE = "../../.assets/data/pelf/temp/"

In [ ]:
pwd

In [ ]:
df_right = pd.read_pickle(os.path.join(DATAPATH_INTERMEDIATE, "data_right_side.pkl.zip"))
df_left = pd.read_pickle(os.path.join(DATAPATH_INTERMEDIATE, "data_left_side.pkl.zip"))

## Constructing the direction label

First we make sure that there is a clear separation between these four directions, so we can actually distinguish the different directions.

In [ ]:
df_right.groupby("nr_of_round").first()["angle_measured"].hist(bins=np.linspace(0, 360, 36), label="Right Side")
df_left.groupby("nr_of_round").first()["angle_measured"].hist(bins=np.linspace(0, 360, 36), label="Left Side")
plt.title("Histogram for both sets")
plt.xlabel("Angle calculated",fontsize=14)
plt.ylabel("# Angles",fontsize=14)
plt.legend(loc="best");

### Categorical encoding

The next thing is to arrive at a categorization of the directions. For this, we make a manual split for the angles into N(orth), S(outh), E(ast), W(est). We will save this label into a new column `dir_label`. As illustrated above, we 
use the first angle of each round to determine in what direction the PELF is going. 

In [ ]:
df_first_angles_right = df_right.drop_duplicates(subset="nr_of_round")[["angle_measured", "nr_of_round"]]
df_first_angles_left = df_left.drop_duplicates(subset="nr_of_round")[["angle_measured", "nr_of_round"]]

df_first_angles_right["dir_label"] = "TBD"
df_first_angles_right["dir_label"] = df_first_angles_right["dir_label"].where(((df_first_angles_right["angle_measured"] > 0) & 
                                                                               (df_first_angles_right["angle_measured"] <= 90)), "E")
df_first_angles_right["dir_label"] = df_first_angles_right["dir_label"].where(((df_first_angles_right["angle_measured"] > 90) & 
                                                                               (df_first_angles_right["angle_measured"] <= 180)), "N")
df_first_angles_left["dir_label"] = "TBD"
df_first_angles_left["dir_label"] = df_first_angles_left["dir_label"].where(((df_first_angles_left["angle_measured"] > 180) & 
                                                   (df_first_angles_left["angle_measured"] <= 270)), "W")
df_first_angles_left["dir_label"] = df_first_angles_left["dir_label"].where(((df_first_angles_left["angle_measured"] > 270) & 
                                                   (df_first_angles_left["angle_measured"] <= 360)), "S")

df_right = df_right.merge(df_first_angles_right.drop("angle_measured", axis=1), on="nr_of_round")
df_left= df_left.merge(df_first_angles_left.drop("angle_measured", axis=1), on="nr_of_round")

In [ ]:
df_right[["nr_of_round", "dir_label"]].sample(5)

In [ ]:
plt.figure(figsize=(10,10))
custom_lines = [plt.Line2D([0], [0], color="red", lw=4),
                plt.Line2D([0], [0], color="blue", lw=4),
                plt.Line2D([0], [0], color="green", lw=4),
                plt.Line2D([0], [0], color="purple", lw=4)]

plt.scatter(df_right[df_right["dir_label"] == "N"]["x_measured"], df_right[df_right["dir_label"] == "N"]["y_measured"], color="red", alpha=0.2)
plt.scatter(df_right[df_right["dir_label"] == "E"]["x_measured"], df_right[df_right["dir_label"] == "E"]["y_measured"], color="blue", alpha=0.2)
plt.scatter(df_left[df_left["dir_label"] == "S"]["x_measured"], df_left[df_left["dir_label"] == "S"]["y_measured"], color="green", alpha=0.2)
plt.scatter(df_left[df_left["dir_label"] == "W"]["x_measured"], df_left[df_left["dir_label"] == "W"]["y_measured"], color="purple", alpha=0.2)

plt.legend(custom_lines, ['N', 'E', 'S','W'])

plt.xlabel("X-Position")
plt.xlabel("Y-Position");

Now that we have the 4 directions separated cleanly we can combine the dataframes for the left and right side to one total dataframe. We can also delete some columns we don't need anymore.

Let's see how many rounds we are making for each direction.

In [ ]:
df_total = pd.concat([df_right, df_left]).reset_index().drop(["motor_position_left", "motor_position_right", "timestamp_transfer", "brightness_difference", "index"], axis=1)
df_total.groupby("dir_label").size()

This means that the data set is not evenly distributed, PELF made most rounds going south from the crossover. This is interesting, because everytime the robot goes south at the crossover it should have gone east on the next crossover. Maybe there was a bug which prevented it from completing the eastern round.

### One-hot encoding

Instead of using the categories directly, we will use a strategy called "one-hot encoding". A simple category for the direction would imply and order between the directions for some algorithms, e.g. N<E<S<W. Of course this would not reflect the actual meaning of the label. Instead of this, each category is will be made an individual column with a value of either one or zero. Only one of those column can be 1 for any row. <br>

In [ ]:
df_total = df_total.join(pd.get_dummies(df_total["dir_label"], prefix='direction'))

In [ ]:
df_total[["nr_of_round", "dir_label", "direction_E", "direction_N", "direction_S", "direction_W"]].sample(5)

## Feature engineering

### Idea

The goal for our machine learning algorithm is to predict the direction of the PELF anywhere on the track after a small time of collecting data (about 3 seconds). Instead of just using the raw data we get after the 3 seconds, we will do some feature engineering.

### Slicing the data 

So far the dataframe contains the full length of the track. But we want to arrive at results faster, i.e. not wait for a full round, which means we need short slices of the data. We will slice the data into parts which would take about 3 seconds to collect in real time: With the currently used sampling rate this would be about 30 data points.

After the next transformation, the dataframe will contain 80 more columns. For every data point we will look at the last 20 data points and calculate the rolling mean over the 10 points before that.

As an example consider an artificial data point with the index 100: The row for this data point will also contain (among many others) a new column called `brightness_left_12`, which contains the mean over the `brightness_left`-value of the data points with index 78-88. This will be done for all four robot transmitted values and for the last 20 data points. 

In [ ]:
print(df_total.columns)

In [ ]:
shift_columns = ["brightness_left", "brightness_right",
                 "motor_speed_left", "motor_speed_right"]

for col in shift_columns:
    for i in range(20):
        df_total[f"{col}_{i}"] = df_total[col].rolling(10).mean().shift(i)

df_total = df_total.drop(["brightness_left",
                          "brightness_right", 
                          "motor_speed_left",
                          "motor_speed_right"], axis=1).dropna()

In [ ]:
df_total.head(5)

## Normalization

Since there is large spread in the size of the values, this will affect the calculation of the gradients in a negative fashion. Therefore, a normalization is needed to shift all values to the range between 0 and 1. Not using normalization will lead to a stronger influence of entries with values at one end of the range (small/big). Normalizing brings these on a more homogeneous range.

In [ ]:
features = [name for name in df_total.columns if ("brightness" in name) | ("motor" in name)]

minmaxer = MinMaxScaler()

df_normalized = df_total.copy()
df_normalized[features] = pd.DataFrame(minmaxer.fit_transform(df_total[features]))
df_normalized.dropna(subset=["brightness_left_1", "motor_speed_right_12"], inplace=True)

For comparison, here is an example of some original values versus the normalized values. 

In [ ]:
comparison_indices = np.random.randint(1000, 10000, size=5)
comparison_column = random.choice(features)

print(df_total.loc[comparison_indices, comparison_column])
print(df_normalized.loc[comparison_indices, comparison_column])

# Saving the data for further processing

In [ ]:
#df_normalized.to_pickle(os.path.join(DATAPATH_INTERMEDIATE, "data_total_normalized.pkl.zip"))

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_